In [1]:
import os 
import sys 
import numpy as np

import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset, random_split

from train import generate_training_examples

import optuna
from optuna.samplers import TPESampler
from optuna.pruners import MedianPruner
import optuna.visualization as vis
import plotly.io as pio

import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages

from abc import ABC, abstractmethod

In [2]:
from model_baseline import SNV2LinFreqs

In [3]:
def add_gaussian_noise(tensor, mean=0, stddev=0.1):
    noise=torch.randn(tensor.size())*stddev+mean
    noised_tensor=tensor+noise
    return noised_tensor

In [3]:
device = (
        "cuda"
        if torch.cuda.is_available()
        else "mps"
        if torch.backends.mps.is_available()
        else "cpu"
    )
print(f"Using {device} device")

n = 10000
known_freqs_file = f'known_freqs_{n}.npy'
snv_freqs_file = f'snv_freqs_{n}.npy'

Using mps device


In [4]:
X=np.load(known_freqs_file)
y=np.load(snv_freqs_file)

In [5]:
known_freqs=torch.from_numpy(X).to(device)
snv_freqs=torch.from_numpy(y).to(device)

In [6]:
dataset = TensorDataset(snv_freqs, known_freqs)

total_size = len(dataset)
test_size = total_size // 10  # 10% for test
validation_size = total_size // 10  # 10% for validation
train_size = total_size - test_size - validation_size  # 80% train

# Split the dataset
train_dataset, validation_dataset, test_dataset = random_split(
        dataset, [train_size, validation_size, test_size]
    )

# verify sizes
print(f"Training set size: {len(train_dataset)}")
print(f"Validation set size: {len(validation_dataset)}")
print(f"Test set size: {len(test_dataset)}")

Training set size: 8000
Validation set size: 1000
Test set size: 1000


In [7]:
batch_size = 1000  # Arbitrarily chosen
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=False)
validation_loader = DataLoader(validation_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [8]:
input_size = snv_freqs.shape[1]
output_size = known_freqs.shape[1]

In [9]:
config = {
        'num_hidden_layers': (1, 5),
        'dropout': (0, 0.5),
        'lr': (1e-5, 1e-1),
        'weight_decay': (1e-5, 1e-1),
        'epochs': 500,
        'patience': 10, 
    }

model = SNV2LinFreqs(train_loader, test_loader, input_size, output_size, device)

# Perform optimization
best_params = model.optimize(n_trials=100, config=config, pruner=True)

print('Best parameters:', best_params)

model.plot_trials(params.prefix)

model.plot_loss(params.prefix)

Using mps device


[I 2024-07-22 21:24:44,190] A new study created in RDB with name: hyper_param_tuning_snv2linfreqs


A new study created in RDB with name: hyper_param_tuning_snv2linfreqs
Using mps device


[I 2024-07-22 21:33:07,502] Trial 0 finished with value: 2.6539233658695593e-05 and parameters: {'num_hidden_layers': 5, 'dropout': 0.13674295813412485, 'weight_decay': 0.0408037389964794, 'lr': 8.438428212593237e-05}. Best is trial 0 with value: 2.6539233658695593e-05.


Trial 0 finished with value: 2.6539233658695593e-05 and parameters: {'num_hidden_layers': 5, 'dropout': 0.13674295813412485, 'weight_decay': 0.0408037389964794, 'lr': 8.438428212593237e-05}. Best is trial 0 with value: 2.6539233658695593e-05.
Using mps device


...

[W 2024-07-23 09:59:31,043] Trial 51 failed with parameters: {'num_hidden_layers': 5, 'dropout': 0.3492118367027861, 'weight_decay': 0.046032161711478635, 'lr': 1.7335012175215987e-05} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "/opt/miniconda3/envs/ww/lib/python3.9/site-packages/optuna/study/_optimize.py", line 196, in _run_trial
    value_or_values = func(trial)
  File "/Users/DannyPang/Desktop/bioinfmsc/diss/model_baseline.py", line 263, in <lambda>
    self.study.optimize(lambda trial: self._objective(trial, config), n_trials=n_trials)
  File "/Users/DannyPang/Desktop/bioinfmsc/diss/model_baseline.py", line 205, in _objective
    train_losses, val_losses = model.fit(
  File "/Users/DannyPang/Desktop/bioinfmsc/diss/model_baseline.py", line 81, in fit
    train_loss += loss.item()
KeyboardInterrupt


Trial 51 failed with parameters: {'num_hidden_layers': 5, 'dropout': 0.3492118367027861, 'weight_decay': 0.046032161711478635, 'lr': 1.7335012175215987e-05} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "/opt/miniconda3/envs/ww/lib/python3.9/site-packages/optuna/study/_optimize.py", line 196, in _run_trial
    value_or_values = func(trial)
  File "/Users/DannyPang/Desktop/bioinfmsc/diss/model_baseline.py", line 263, in <lambda>
    self.study.optimize(lambda trial: self._objective(trial, config), n_trials=n_trials)
  File "/Users/DannyPang/Desktop/bioinfmsc/diss/model_baseline.py", line 205, in _objective
    train_losses, val_losses = model.fit(
  File "/Users/DannyPang/Desktop/bioinfmsc/diss/model_baseline.py", line 81, in fit
    train_loss += loss.item()
KeyboardInterrupt


[W 2024-07-23 09:59:31,044] Trial 51 failed with value None.


Trial 51 failed with value None.


KeyboardInterrupt: 

In [12]:
study_name='hyper_param_tuning_snv2linfreqs'

study = optuna.create_study(study_name=study_name, 
                            storage=f'sqlite:///{study_name}.db', 
                            load_if_exists=True)

[I 2024-07-23 11:53:08,950] Using an existing study with name 'hyper_param_tuning_snv2linfreqs' instead of creating a new one.


Using an existing study with name 'hyper_param_tuning_snv2linfreqs' instead of creating a new one.
